# Entscheidungsbaum-Klassifikation mit dem Iris-Datensatz
In diesem Notebook wird ein **Entscheidungsbaum** mit dem Iris-Datensatz
trainiert, evaluiert und visualisiert. Der Fokus liegt auf Interpretierbarkeit
und praktischer Anwendung klassischer ML-Methoden.

## Bibliotheken importieren

In [ ]:
import pandas as pd

## Daten laden

In [ ]:
file_path = '../data/processed/iris.csv'
df = pd.read_csv(file_path, sep=',')

## Feature-Ziel-Trennung

In [ ]:
features = [c for c in df.columns if c != 'species']
X = df[features]
y = df['species']

## Aufteilen in Trainings- und Testdaten

In [ ]:
from sklearn.model_selection import train_test_split
X_tr, X_tst, y_tr, y_tst = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

## Entscheidungsbaum definieren und trainieren

In [ ]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
model.fit(X_tr, y_tr)

## Vorhersage auf Testdaten

In [ ]:
y_predicted = model.predict(X_tst)

## Konfusionsmatrix erstellen

In [ ]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_tst, y_predicted)
print('Confusion Matrix:')
print(matrix)

## Klassifikationsbericht

In [ ]:
from sklearn.metrics import classification_report
report = classification_report(y_tst, y_predicted, digits=3)
print('Classification Report:')
print(report)

## Visualisierung des Entscheidungsbaums

In [ ]:
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 8))
plot_tree(
    model,
    feature_names=features,
    class_names=model.classes_,
    filled=True,
    rounded=True,
    fontsize=6
)
plt.show()
plt.savefig('../figs/baumstruktur.png', dpi=300, bbox_inches='tight')

### Hinweise zur Interpretation
- Jeder Knoten enthält das Splitting-Kriterium und die Klassenzugehörigkeit
- Die Färbung spiegelt die Dominanz der jeweiligen Klasse wider
- Je höher die Diagonaleinträge in der Confusion Matrix, desto besser die Klassifikation
- `ccp_alpha` kann verwendet werden, um Pruning zu steuern und Overfitting zu vermeiden